# NFL Baseline
- create target_df (distance in tracking_df is lower than threshold=3)
https://www.kaggle.com/code/stgkrtua/nfl-creatatraindataset-targetdf
- create dataset save frames in target_df
https://www.kaggle.com/code/stgkrtua/nfl-createdataset-saveframes
- check saved images
https://www.kaggle.com/code/stgkrtua/nfl-checkdataset-plotsavedimage

# import libraries

In [1]:
# general
import os
import gc
import pickle
import glob
import random
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import time
import math
from tqdm.notebook import tqdm

import sys
# warningの表示方法の設定
import warnings
warnings.filterwarnings("ignore")

# Set Configurations

In [2]:
CFG = {
        "kaggle" : False,
        "DEBUG" : False,
        "img_size" : (224, 224),
        "masksize_helmet_ratio" : 4,
}
if CFG["DEBUG"]:
        CFG["sample_num"] = 1000
else:
        CFG["sample_num"] = -1
CFG["INPUT_DIR"] = "/workspace/input"
CFG["OUTPUT_DIR"] = "/workspace/output"
CFG["TRAIN_HELMET_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_baseline_helmets.csv")
CFG["TRAIN_TRACKING_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_player_tracking.csv")
CFG["TRAIN_VIDEO_META_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_video_metadata.csv")
CFG["TRAIN_LABEL_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_labels.csv")
CFG["TARGET_CSV"] = os.path.join(CFG["INPUT_DIR"], "target_fillna0_shift_2.csv")
CFG["TRAIN_E_IMG_DIR"] = os.path.join(CFG["INPUT_DIR"], "train_frames")
CFG["TRAIN_S_IMG_DIR"] = CFG["TRAIN_E_IMG_DIR"]
CFG["CONTACT_IMG_DIR"] = os.path.join(CFG["INPUT_DIR"], "contact_images")

## Dataset Utils

In [3]:
def set_inimg_window(crop_pos, mask_size, img_size=(720, 1280)):#crop_pos = [left, top, right, bot]
    if mask_size[1] >= img_size[0]:
        top, bot = 0, img_size[1]
    else:
        top=(crop_pos[1] + crop_pos[3])//2 - mask_size[1]//2
        bot=(crop_pos[1] + crop_pos[3])//2 + mask_size[1]//2
        if top < 0:
            bot = bot - top
            top = 0
        elif bot > img_size[0]:
            top = top - (bot-img_size[0])
            bot = img_size[0]

    if mask_size[0] >= img_size[1]:
        left, right = 0, img_size[1]
    else:
        left = (crop_pos[0] + crop_pos[2])//2 - mask_size[0]//2
        right = (crop_pos[0] + crop_pos[2])//2 + mask_size[0]//2
        if left < 0:
            right = right - left
            left = 0
        elif right > img_size[1]:
            left = left - (right - img_size[1])
            right = img_size[1]
    crop_area = np.array([left, top, right, bot]).astype(np.int)
    return crop_area

In [4]:
def get_crop_area(p1_helmet, p2_helmet, input_size=(720, 1280)):#helmet[left, width, top, height]
    if (p1_helmet[1]==0 and p1_helmet[3]==0) and (p2_helmet[1]==0 and p2_helmet[3]==0):
        crop_area = [0, 0, input_size[1], input_size[0]]
        # print("bose player's helmet is not detected.")
        return crop_area
    elif (p2_helmet[1]==0 and p2_helmet[3]==0) and (p1_helmet[1] != 0 and p1_helmet[3]!=0):
        # print("p1 detected.")
        crop_x_center, crop_y_center = p1_helmet[0] + (p1_helmet[1])//2, p1_helmet[2] + (p1_helmet[3])//2
        helmet_base_size = (p1_helmet[1] + p1_helmet[3])*0.5*CFG["masksize_helmet_ratio"]*4
        output_size = [helmet_base_size, helmet_base_size]
    elif (p1_helmet[1]==0 and p1_helmet[3]==0) and (p2_helmet[1]!=0 and p2_helmet[3]!=0):
        # print("p2 detected.")
        crop_x_center, crop_y_center = p2_helmet[0] + (p2_helmet[1])//2, p2_helmet[2] + (p2_helmet[3])//2
        helmet_base_size = (p2_helmet[1] + p2_helmet[3])*0.5*CFG["masksize_helmet_ratio"]*4
        output_size = [helmet_base_size, helmet_base_size]
    else:
    #     print("p1 and p2 detected.")
        p1_x_center, p1_y_center = p1_helmet[0] + (p1_helmet[1])//2, p1_helmet[2] + (p1_helmet[3])//2
        p2_x_center, p2_y_center = p2_helmet[0] + (p2_helmet[1])//2, p2_helmet[2] + (p2_helmet[3])//2
        crop_x_center, crop_y_center = (p1_x_center + p2_x_center)//2, (p1_y_center + p2_y_center)//2
        helmet_base_size = (abs(p1_x_center - p2_x_center) + abs(p1_y_center - p2_y_center))*0.5 \
                            + ((p1_helmet[1] + p2_helmet[1])*0.5 + (p1_helmet[3] + p2_helmet[3])*0.5)*0.5*CFG["masksize_helmet_ratio"]*2
        output_size = [helmet_base_size, helmet_base_size]
    
    # print("crop center", crop_x_center, crop_y_center)
    crop_left = crop_x_center - output_size[1]//2
    crop_top = crop_y_center - output_size[0]//2
    crop_right = crop_x_center + output_size[1]//2
    crop_bot = crop_y_center + output_size[0]//2
    crop_area = [crop_left, crop_top, crop_right, crop_bot]
    crop_area = set_inimg_window(crop_area, output_size)
    return crop_area

In [5]:
def get_playermasked_img(img, helmet_pos, img_size=(720, 1280, 3)):#helmet pos = [left, width, top, height]
    if helmet_pos[1] == 0 and helmet_pos[3] == 0:
        mask = np.ones_like(img)
        return mask
    mask_size=(helmet_pos[1]+helmet_pos[3])*0.5*CFG["masksize_helmet_ratio"]# helmetの大きさによってplayerの範囲も変更
    helmet_area = [helmet_pos[0], helmet_pos[2], helmet_pos[0]+helmet_pos[1], helmet_pos[2]+helmet_pos[3]]#[left, top, right, bot]
    player_area = set_inimg_window(helmet_area, (mask_size,mask_size))
    mask = np.zeros(img_size, dtype=np.float)
    cv2.rectangle(mask, [player_area[0], player_area[1]], [player_area[2], player_area[3]], (255, 255, 255), -1)
    mask = np.clip(mask, 0, 1).astype(np.float)
    return mask

# Load Data

In [6]:
target_df = pd.read_csv(CFG["TARGET_CSV"])
if CFG["DEBUG"]:
    target_df = target_df.sample(CFG["sample_num"])
elif CFG["sample_num"] != -1:
    target_df = target_df.sample(CFG["sample_num"])
print(len(target_df))
display(target_df["contact"].value_counts())

660553


0    596241
1     64312
Name: contact, dtype: int64

In [7]:
for view in ["Endzone", "Sideline"]:
    file_id_tmp = None
    start_time = time.time()
    for idx in range(len(target_df)):
        print("\r {}/{}".format(idx, len(target_df)), end="")
        target_info = target_df.iloc[idx]
        # read frame image
        game_play = target_info.game_play
        frame = target_info.frame
        contact_id = target_info.contact_id
        file_id = f"{game_play}_{view}_{frame:04}.jpg"
        if file_id != file_id_tmp:
            filename = os.path.join(CFG["TRAIN_E_IMG_DIR"], file_id)
            img = cv2.imread(filename)
            if not img is None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                view_tmp = view
                file_id_tmp = file_id
            else:
                print(f"\t\t {file_id} does not exists.")
        else:
            # player highlight mask
            player1 = target_info.nfl_player_id_1
            player2 = target_info.nfl_player_id_2
            if view == "Endzone":
                p1_helmet = np.array([target_info.E_left_1, target_info.E_width_1,
                                    target_info.E_top_1, target_info.E_height_1]).astype(np.int)
                p2_helmet = np.array([target_info.E_left_2, target_info.E_width_2,
                                    target_info.E_top_2, target_info.E_height_2]).astype(np.int)
            elif view == "Sideline":
                p1_helmet = np.array([target_info.S_left_1, target_info.S_width_1,
                                    target_info.S_top_1, target_info.S_height_1]).astype(np.int)
                p2_helmet = np.array([target_info.S_left_2, target_info.S_width_2,
                                    target_info.S_top_2, target_info.S_height_2]).astype(np.int)
            mask_img = img.copy()
            mask1 = get_playermasked_img(mask_img, p1_helmet)# helmet=[left, width, top, height]
            mask2 = get_playermasked_img(mask_img, p2_helmet)
            mask = np.clip(mask1 + mask2, 0, 1).astype(np.float)
            mask_img = mask*mask_img
            # crop players area
            crop_area = get_crop_area(p1_helmet, p2_helmet)# crop_area=[left, top, right, bot]

            mask_img = mask_img[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2], :]
            mask_img = cv2.resize(mask_img, dsize=CFG["img_size"])
            
            contact_fileid = f"{contact_id}_{view}.jpg"
            contact_filename = os.path.join(CFG["CONTACT_IMG_DIR"], contact_fileid)
            cv2.imwrite(contact_filename, mask_img)
        if idx%10000 == 0:
            elapsed_time = (time.time() - start_time)/60
            print(f"\t\t idx = {idx}, elapsed time = {elapsed_time:.3f} min")


 0/660553		 idx = 0, elapsed time = 0.000 min
 10000/660553		 idx = 10000, elapsed time = 3.511 min
 20000/660553		 idx = 20000, elapsed time = 7.015 min
 25001/66055358180_000986_Endzone_1046.jpg does not exists.
 25002/66055358180_000986_Endzone_1046.jpg does not exists.
 25003/66055358180_000986_Endzone_1046.jpg does not exists.
 25004/66055358180_000986_Endzone_1046.jpg does not exists.
 25005/66055358180_000986_Endzone_1046.jpg does not exists.
 25006/66055358180_000986_Endzone_1046.jpg does not exists.
 25007/66055358180_000986_Endzone_1046.jpg does not exists.
 25008/66055358180_000986_Endzone_1046.jpg does not exists.
 25009/66055358180_000986_Endzone_1046.jpg does not exists.
 25010/66055358180_000986_Endzone_1046.jpg does not exists.
 25011/66055358180_000986_Endzone_1046.jpg does not exists.
 25012/66055358180_000986_Endzone_1046.jpg does not exists.
 25013/66055358180_000986_Endzone_1046.jpg does not exists.
 25014/66055358180_000986_Endzone_1046.jpg does not exists.
 25015